In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.utils import to_categorical
import numpy as np
import keras_tuner as kt

# Load the dataset
data = pd.read_csv("dataset.csv")

# Extract features and labels
image_paths = data["image_path"]
labels = data["artist_label"]

# Function to load and preprocess images
def load_and_preprocess_images(image_paths, target_size=(128, 128)):
    images = []
    for img_path in image_paths:
        img = image.load_img(img_path, target_size=target_size)
        img_array = image.img_to_array(img)
        images.append(img_array)
    
    return np.array(images)

# Load and preprocess images
X = load_and_preprocess_images(image_paths)
X = X / 255.0  # Normalize the images to [0, 1] range

# Convert labels to numpy array and one-hot encode them
y = np.array(labels)
y = to_categorical(y, num_classes=3)  # 3 classes (0, 1, 2)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the model-building function for hyperparameter tuning
def build_model(hp):
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(
            hp.Int('conv_1_filters', min_value=32, max_value=128, step=32), 
            (3, 3), activation='relu', input_shape=(128, 128, 3)
        ),
        tf.keras.layers.MaxPooling2D(2, 2),
        
        tf.keras.layers.Conv2D(
            hp.Int('conv_2_filters', min_value=64, max_value=256, step=64), 
            (3, 3), activation='relu'
        ),
        tf.keras.layers.MaxPooling2D(2, 2),
        
        tf.keras.layers.Conv2D(
            hp.Int('conv_3_filters', min_value=128, max_value=512, step=128), 
            (3, 3), activation='relu'
        ),
        tf.keras.layers.MaxPooling2D(2, 2),
        
        tf.keras.layers.Flatten(),
        
        tf.keras.layers.Dense(
            hp.Int('dense_units', min_value=128, max_value=512, step=128), 
            activation='relu'
        ),
        tf.keras.layers.Dropout(
            hp.Float('dropout_rate', min_value=0.2, max_value=0.5, step=0.1)
        ),
        
        tf.keras.layers.Dense(3, activation='softmax')  # 3 classes
    ])
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            learning_rate=hp.Float('learning_rate', min_value=1e-5, max_value=1e-2, sampling='LOG')
        ),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Initialize the tuner
tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=10,
    hyperband_iterations=2,
    directory='hyperparameter_tuning',
    project_name='cnn_tuning'
)

# Search for the best hyperparameters
tuner.search(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Get the best model and hyperparameters
best_model = tuner.get_best_models(num_models=1)[0]
best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"Best Hyperparameters: {best_hp.values}")

# Evaluate the best model
test_loss, test_acc = best_model.evaluate(X_test, y_test)
print(f"Test accuracy of the best model: {test_acc}")

# Predict on the test set
y_pred = np.argmax(best_model.predict(X_test), axis=1)  # Convert softmax probabilities to class labels

# Get the actual class labels (as integers)
y_true = np.argmax(y_test, axis=1)

# Classification report for performance evaluation
report = classification_report(y_true, y_pred, target_names=['Class 0', 'Class 1', 'Class 2'])
print(report)


Trial 50 Complete [00h 01m 34s]
val_accuracy: 0.8172484636306763

Best val_accuracy So Far: 0.9240246415138245
Total elapsed time: 01h 49m 08s

Search: Running Trial #51

Value             |Best Value So Far |Hyperparameter
96                |128               |conv_1_filters
256               |64                |conv_2_filters
128               |128               |conv_3_filters
256               |256               |dense_units
0.2               |0.4               |dropout_rate
0.00026403        |0.00035343        |learning_rate
4                 |10                |tuner/epochs
0                 |0                 |tuner/initial_epoch
1                 |0                 |tuner/bracket
0                 |0                 |tuner/round

Epoch 1/4
57/61 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - accuracy: 0.5446 - loss: 0.8853